# **Algorithm for Topic Extraction Using LDA**

This document present the process of development of an unsupervised algorithm for topic extraction. Currently the most reliable technic is the LDA (Latent Dirichlet Allocation) algorithm, which is base on the Dirichlet distribution.

*This notebook is meant to save the important notes and the project decisions. However, a ".py" file containing the same code is available on this folder.*

## **Approach**

## Input the Data

#### The first step is to make the input of the data and the filter by language to avoid inconsistent results

In [1]:
from Algorithms import preProcessing, modelUsageAPI

/home/diego/Documentos/anaconda3/envs/i2a2/lib/python3.7/site-packages/tqdm/_tqdm.py:634: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
paths = ['datasets/articles_bbc_2018_01_30.csv', 'datasets/transcripts.csv', 'datasets/topics.csv']
targets = ['articles','transcript', 'question_text']

In [3]:
data = preProcessing.import_files(paths, targets,
                                   preCleaning = True,
                                   dropna = 'index',
                                   verbose = True)

Loading the dataset 0.

Input Format:
Rows: 309, Columns: 2.

Pre cleaning format:
Rows: 308, Columns: 2
Loading the dataset 1.

Input Format:
Rows: 2467, Columns: 2.

Pre cleaning format:
Rows: 2467, Columns: 2
Loading the dataset 2.

Input Format:
Rows: 5000, Columns: 3.

Pre cleaning format:
Rows: 5000, Columns: 3

Removing unwanted information using targets.


In [4]:
data.shape

(7775, 1)

In [5]:
preProcessing.language_detection(data,
                                 verbose = True)


en    7722
fr       9
fa       9
id       5
ru       4
hi       4
uk       4
ar       4
vi       4
sw       3
pt       2
tr       2
es       2
de       1
Name: lang, dtype: int64

Most Frequent language: en    7722
Name: lang, dtype: int64


en    7722
Name: lang, dtype: int64

In [6]:
data = preProcessing.language_cleaning(dataFile = data,
                                        language = 'en',
                                        verbose = True)

Cleaning data using 'en' language.


## Pre Processing the data

#### **Tokenizing** the documents to the word level.

In [7]:
data = preProcessing.tokenization(data, level = 'word', verbose = True)

Tokenizing data to word
.


#### Preparing for **Lemmatizizing** using POS tagging.

In [8]:
data = preProcessing.POS_tagging(data)

In [9]:
data = preProcessing.lemmatizing(data)

#### Removing **StopWords** using the english stopwords from the Natural Language Toolkit (NLTK) and removing any token less than 2 characthers.

In [10]:
data = preProcessing.removeStopWords(data, minSize = 2)

## Trainning the LDA Model

### Preparing the Data

#### Generating the tokens using the Bigram and the Trigram Model

In [11]:
tokens = preProcessing.Bi_n_TrigramModel(data, min_cnt = 1, verbose = True)

Getting tokens From data file and converting into a list of tokens.
Building the Bigram Model
Building the Trigram Model
Importing the Trigram Model and converting into list


#### Creating the dictionary using the the tokens

In [12]:
dictionary = preProcessing.generateDictionary(tokens, min_thld = 3, verbose = True)
modelUsageAPI.save(dictionary,'models/dictionary.dict')

Generating the Ditionary.
Filtering dictionary using the minimun threshold: 3


#### Generating the BOW for the LDA model

In [13]:
bow = preProcessing.generateBOW(dictionary, tokens, verbose = True)

Generating Bag Of Words.


#### Generating the LDA based model

In [14]:
numberOfTopics = 14
%time LDAModel = preProcessing.trainModel(bow, dictionary, numTopics = numberOfTopics, numPasses = 4, verbose = True)

Trainning LDA model using the inputed BOW and Dictionary.
Parameters: Topics: 14, Passes: 4
CPU times: user 1min 1s, sys: 23.4 s, total: 1min 25s
Wall time: 48.7 s


#### A quick view on the topics

In [15]:
for i,topic in LDAModel.show_topics(formatted=True, num_topics = numberOfTopics, num_words=20):
    print(str(i)+": "+ topic)
    print()

0: 0.022*"site" + 0.014*"book" + 0.009*"story" + 0.008*"mall" + 0.007*"film" + 0.006*"video" + 0.006*"tell" + 0.005*"laughter" + 0.005*"waterproof" + 0.005*"read" + 0.005*"ring" + 0.005*"call" + 0.005*"lie" + 0.005*"location" + 0.005*"cable" + 0.005*"thing" + 0.005*"think" + 0.004*"happen" + 0.004*"find" + 0.004*"many"

1: 0.014*"look" + 0.010*"water" + 0.010*"planet" + 0.009*"find" + 0.009*"earth" + 0.008*"animal" + 0.008*"time" + 0.007*"life" + 0.006*"think" + 0.006*"ocean" + 0.006*"specie" + 0.006*"back" + 0.006*"place" + 0.005*"thing" + 0.005*"live" + 0.005*"big" + 0.005*"way" + 0.004*"could" + 0.004*"show" + 0.004*"first"

2: 0.007*"country" + 0.007*"cancer" + 0.006*"today" + 0.005*"farmer" + 0.005*"oil" + 0.005*"energy" + 0.005*"look" + 0.005*"year" + 0.005*"government" + 0.005*"percent" + 0.004*"world" + 0.004*"system" + 0.004*"think" + 0.004*"let" + 0.004*"could" + 0.004*"problem" + 0.004*"time" + 0.004*"food" + 0.004*"good" + 0.004*"market"

3: 0.015*"people" + 0.008*"think" +

In [16]:
modelUsageAPI.save(LDAModel,'models/LDAmodelExtended.pkl')

## Validating the Model

#### Validating the model on the document in position 0.

In [17]:
nbDoc = 0
valDoc = data.articles.loc[nbDoc]

In [18]:
print(valDoc[:500])

Image copyright PA/EPA Image caption Oligarch Roman Abramovich (l) and PM Dmitry Medvedev are on the list

Russian President Vladimir Putin says a list of officials and businessmen close to the Kremlin published by the US has in effect targeted all Russian people.

The list names 210 top Russians as part of a sanctions law aimed at punishing Moscow for meddling in the US election.

However, the US stressed those named were not subject to new sanctions.

Mr Putin said the list was an unfr


In [19]:
LDAModel[bow[nbDoc]]

[(11, 0.49627098), (13, 0.50307363)]

In [20]:
modelUsageAPI.infer('Document.txt','models/LDAmodelExtended.pkl','models/dictionary.dict' )

Loading the Document from path: Document.txt
Loading model from path: models/LDAmodelExtended.pkl
Loading dictionary from path: models/dictionary.dict
Inferring:



,topic #,weight,words in topic
0,3,0.08,"0.015*""people"" + 0.008*""think"" + 0.008*""countr..."
1,8,0.11,"0.012*""laughter"" + 0.011*""thing"" + 0.011*""look..."
2,9,0.03,"0.036*""woman"" + 0.019*""country"" + 0.018*""schoo..."
3,11,0.68,"0.012*""people"" + 0.012*""think"" + 0.011*""thing""..."
4,13,0.10,"0.029*""people"" + 0.014*""think"" + 0.008*""laught..."
